In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split , KFold
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.preprocessing import StandardScaler
import optuna
from optuna.integration import LightGBMPruningCallback

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample.csv")

In [ ]:
def preprocess(df):
  df.index = pd.to_datetime(df["datetime"])
  df = df["2014-05-01":]
  df = df.reset_index(drop=True)
  
  def weather(x):
    if x.find("快晴"):
      x = 0
    elif x.find("晴れ"):
      x = 1
    elif x.find("曇"):
      x = 2
    elif x.find("薄曇"):
      x = 3
    elif x.find("雨"):
      x = 4
    else:
      x = 5
    return x

  df["days"] = df.index
  df["precipitation"] = df["precipitation"].apply(lambda x : -1 if x=="--" else x).astype(np.float64)
  df["fun"] = df["remarks"].apply(lambda x: 1 if x=="お楽しみメニュー" else 0)
  df["curry"] = df["name"].apply(lambda x : 1 if x.find("カレー")>=0 else 0)
  df["weather"] = df["weather"].apply(weather)
  #df["subuta"] = df["name"].apply(lambda x : 1 if x.find("酢豚")>=0 else 0)
  #df["sp"] = df["remarks"].apply(lambda x : 1 if x==("スペシャルメニュー")>=0 else 0)
  #df["kd"] = df["remarks"].apply(lambda x : 1 if x==("料理長")>=0 else 0)
  
  categorical_cols = ["precipitation", "weather", "days", "fun", "curry", "week", "name", "remarks", "event"]
  
  for cols in categorical_cols:
    if cols in df.columns:
      df[cols] = df[cols].astype("category")
  return df
 

In [ ]:
df_train = preprocess(train)
test["y"] = 0
df_test = preprocess(test)
cols = ["datetime","weather","days","fun","curry","y"]

In [ ]:
df_train["t"] = 1
df_test["t"] = 0
dat = pd.concat([df_train,df_test],sort=True).reset_index(drop=True)